 Scenarios:
 
1. Find the distinct job titles in the dataset.
2. Find the below for each of the job titles
	2.1 Avg salary
	2.2 Max salary
	2.3 Min salary
3. Country wise titles and avg and max salary.
4. Gender diversity country wise
5. country wise - age group of working population.

Usage:
1. Aggreate functions 
    1.1 avg
    1.2 min
    1.3 max
2. Window functions
    2.1 group by
    2.2 order by
3. filter() or where()
4. Addition of column using withColumn() function

In [2]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import functions
import pyspark.sql.functions as F
from pyspark.sql.window import Window
import sys

In [3]:
if __name__ == "__main__" :
    spark=(SparkSession.builder
                      .appName("Prg4_DF_manipulation_2")
                      .getOrCreate())

23/05/07 15:51:45 WARN Utils: Your hostname, hps17 resolves to a loopback address: 127.0.1.1; using 192.168.236.132 instead (on interface ens33)
23/05/07 15:51:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/07 15:51:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
parquet_dataset="/home/hari/Downloads/LearningSparkV2-master/parquet_files/userdata1.parquet"

In [5]:
df=spark.read.format("parquet").load(parquet_dataset)

# Sc 1: Find the distinct job titles in the dataset.

In [6]:
df.select("title").where((F.trim("title")!='')).orderBy("title",ascending=True).show(truncate=False)
print('Count of distinct titles',df.select("title").distinct().count())

+-------------------------+
|title                    |
+-------------------------+
|61871.37                 |
|Account Coordinator      |
|Account Coordinator      |
|Account Coordinator      |
|Account Coordinator      |
|Account Executive        |
|Account Executive        |
|Account Executive        |
|Account Executive        |
|Account Executive        |
|Account Executive        |
|Account Executive        |
|Account Executive        |
|Account Executive        |
|Account Representative I |
|Account Representative I |
|Account Representative II|
|Account Representative II|
|Accountant I             |
|Accountant I             |
+-------------------------+
only showing top 20 rows

Count of distinct titles 182


# Find the below for each of the job titles - country wise
	2.1 Avg salary
	2.2 Max salary
	2.3 Min salary
    2.4 Use of aliass function
    2.5 Trim function to cleanse
    2.6 agg() for multiple aggregate function use
    2.7 SEQUENCE ID and RANK based on avg salary using withColumn fn (rank and row_number)
    

Notes:
We need to use agg() when we want to use multiple aggreate function
like,
       .agg(F.max("salary"),F.min("salary"),F.avg("salary"))

In [7]:
df.select(F.max("salary"),F.min("salary"),F.avg("salary")).show()
win=(Window.partitionBy("title").orderBy(F.col("Avg_Salary").desc()))
seq=(Window.orderBy("title","rank"))
df_stat_1=(df.where(F.trim("title")!='')
     .orderBy("title")
     .groupBy("title","country")
     .agg(F.max("salary").alias("Max_Salary"),F.min("salary").alias("Min_Salary"),F.avg("salary").alias("Avg_Salary"))
     #.row_number().over(win)
     #.show(truncate=False)
)

(df_stat_1.withColumn("rank",F.rank().over(win))
 .withColumn("Seq_nm",F.row_number().over(seq))
 .show()
)

+-----------+-----------+------------------+
|max(salary)|min(salary)|       avg(salary)|
+-----------+-----------+------------------+
|  286592.99|   12380.49|149005.35665236053|
+-----------+-----------+------------------+



23/05/07 15:52:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/07 15:52:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/07 15:52:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/07 15:52:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+----------------+----------+----------+-----------------+----+------+
|               title|         country|Max_Salary|Min_Salary|       Avg_Salary|rank|Seq_nm|
+--------------------+----------------+----------+----------+-----------------+----+------+
|            61871.37|        "Bonaire|      null|      null|             null|   1|     1|
| Account Coordinator|         Nigeria| 193301.99| 193301.99|        193301.99|   1|     2|
| Account Coordinator|           Japan|  83759.68|  83759.68|         83759.68|   2|     3|
| Account Coordinator|     Philippines|  44739.92|  44739.92|         44739.92|   3|     4|
| Account Coordinator|         Finland|  16468.24|  16468.24|         16468.24|   4|     5|
|   Account Executive|   Aland Islands| 259912.24| 259912.24|        259912.24|   1|     6|
|   Account Executive|           China| 175184.97|  149849.0|       162516.985|   2|     7|
|   Account Executive|          Canada| 100606.14| 100606.14|        100606.14| 

# Sc 3: Age group calculation

3.1 Use current_timestamp to fetch current date
3.2 to_date to convert string into date of specify format
3.3 datediff - get date difference 2 dates
3.4 months_between - get month between two dates
3.5 round () function usage
3.6 current_time() function
3.7 current_timestamp function
3.8 extract day, month and year
3.9 max and min of date

In [50]:
df.select("birthdate").distinct()
((df.select(F.year(F.to_date(("birthdate"),format="MM/d/yyyy")))).filter(F.trim("birthdate")!='').distinct())

df_dt=((df.select((F.to_date(("birthdate"),format="MM/d/yyyy")).alias("dob_dt"))).filter(F.trim("birthdate")!='').distinct())

df_dt.select(F.max("dob_dt"),F.min("dob_dt")).show()

+-----------+-----------+
|max(dob_dt)|min(dob_dt)|
+-----------+-----------+
| 2001-01-10| 1955-01-19|
+-----------+-----------+



In [31]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
(df.select("id","birthdate","gender")
 .filter(F.trim("birthdate")!='')
 .withColumn("dob",F.to_date(F.trim("birthdate"),format="MM/d/yyyy"))
 .withColumn("age",F.round(((F.datediff(F.current_date(),"dob"))/F.lit(365)),2))
 .withColumn("dob_year",F.year("dob"))
 .show()
)
df.withColumn("date",F.current_date()).show() #2023-05-07


+---+----------+------+----------+-----+--------+
| id| birthdate|gender|       dob|  age|dob_year|
+---+----------+------+----------+-----+--------+
|  1|  3/8/1971|Female|1971-03-08| 52.2|    1971|
|  2| 1/16/1968|  Male|1968-01-16|55.34|    1968|
|  3|  2/1/1960|Female|1960-02-01| 63.3|    1960|
|  4|  4/8/1997|Female|1997-04-08| 26.1|    1997|
|  6| 2/25/1983|Female|1983-02-25|40.22|    1983|
|  7|12/18/1987|  Male|1987-12-18|35.41|    1987|
|  8|  3/1/1962|  Male|1962-03-01|61.22|    1962|
|  9| 3/27/1992|  Male|1992-03-27|31.13|    1992|
| 10| 1/28/1997|Female|1997-01-28|26.29|    1997|
| 12| 8/12/1968|Female|1968-08-12|54.77|    1968|
| 13| 8/15/1975|  Male|1975-08-15|47.76|    1975|
| 14| 6/27/1970|Female|1970-06-27| 52.9|    1970|
| 15|12/20/1989|Female|1989-12-20| 33.4|    1989|
| 17| 4/13/1990|Female|1990-04-13|33.09|    1990|
| 18| 1/15/1978|  Male|1978-01-15|45.34|    1978|
| 21|  6/5/1985|Female|1985-06-05|37.95|    1985|
| 23| 1/23/1971|  Male|1971-01-23|52.32|    1971|
